In [1]:
# imports
import pandas as pd
import numpy as np
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcpy.sa import *
import matplotlib.pyplot as plt

In [9]:
# functions
def roi(df, states, state_col = 'STATE_ABBR'):
    """
    Returns Dataframe with only roi data
    Params: Dataframe to filter; list of interested states; name of column where state info is stored
    """
    new_df = df[df[state_col].isin(states)]
    return new_df

def landcover(df, land_type = 82):
    """
    Returns a df with only the specified landcover (default in cultivated crops)
    """
    filtered_df = df[df['landcover'] == land_type]
    return filtered_df

def valid(df, split, ID: str, col = 'Random', seed = 1):
    """
    Makes a random subset of the data for validation
    Params: Dataframe to filter; for split input of 20 yields 80:20 testing:validation split; name of column to be randomized
    """
    if seed !=1:
        np.random.seed(seed)
        
    df[col] = np.random.randint(0,100, df.shape[0])
    testing_df = df.loc[df[col] >= split].copy()
    validation_df = df.loc[df[col] < split].copy()
    testing_path = arcpy.env.workspace+'/test'+ID+'.csv'
    validing_path = arcpy.env.workspace+'/valid'+ID+'.csv'
    testing_df.to_csv(testing_path) 
    validation_df.to_csv(validing_path)
    return testing_path, validing_path

def setup(df, states, split, ID: str):
    """
    Synopsis of roi, valid, and landcover to get testing and validation df's in roi with target land cover types.
    """
    new_df = roi(df, states)
    filtered_df = landcover(new_df)
    return valid(filtered_df, split, ID)

def toXY(in_table:str, out_points: str, ID: str):
    """
    Returns XY points to run SpatialJoin and convert csv to points
    """
    x_coords = "longitude_decimal_degrees"
    y_coords = "latitude_decimal_degrees"
    out_feature_class = out_points + ID

    # Make the XY event layer...
    arcpy.management.XYTableToPoint(in_table, out_feature_class,
                                    x_coords, y_coords)
    return out_feature_class

def density(feat:str, join_features: str, r:int, out_put: str, ID:str):
    """
    Returns edited feature class to be interpolated. Counts the density within certain radius, r.
    Density, in this case, is defined as the number of points within a certain radius, r.
    Params: Feature class from SpatialJoin, feat; r is the search radius
    """
    # set the local variables
    target_features = feat
    out_feature_class = out_put + ID

    arcpy.analysis.SpatialJoin(target_features, 
                               join_features,
                               out_feature_class, 
                               join_operation = "JOIN_ONE_TO_ONE",
                               join_type = "KEEP_ALL",
                               match_option = "WITHIN_A_DISTANCE",
                               search_radius = str(r)+" Kilometers")
    return(out_feature_class)
    
def EBK(in_features: str, output: str, ID: str, z_field: str = "ph_h2o"):
    """
    Returns pH raster across lower 48. 
    """
    out_raster = output + ID

    arcpy.ga.EmpiricalBayesianKriging(in_features,
                                      z_field,
                                      out_raster)
    return out_raster

def toRaster(ga_layer:str, output: str, ID: str):
    """
    Converts EBK to raster.
    """
    in_geostat_layer = ga_layer
    out_raster = output + ID
    arcpy.ga.GALayerToRasters(in_geostat_layer, out_raster)
    return out_raster

def ExtractToPoints(points: str, raster: str, output: str, ID: str):
    """
    Returns points for validation with their predicted pH from EBK raster.
    """
    ExtractValuesToPoints(points, raster, output)
    return output

In [11]:
# set workspace
arcpy.env.workspace = r"C:\path_to_your.gdb"
arcpy.env.overwriteOutput = True

# get NCSS and set ID (to keep track)
ncss_path = "NCSScompleter"
NCSS = pd.DataFrame.spatial.from_featureclass(ncss_path) 
NCSS.rename(columns={'RASTERVALU': 'landcover', 'PageNumber': 'CellID'}, inplace=True)
ID = '6272'

# IMPORTANT VARIABLES

# set roi
# cornbelt = ['IA', 'MO', 'IL', 'IN', 'OH']
states = NCSS['STATE_ABBR']
nation = states.unique()
# set desired validation split
split = 34
# set radius (km)
radius = 0.5

In [12]:
# makes datasets
test_path, valid_path = setup(NCSS, nation, split, ID)

[9]:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
testing_asXY = toXY(test_path, "testing_points", ID)
validation_asXY = toXY(valid_path, "validation_points", ID)

In [14]:
# it's time to krig.
ebk_totest = EBK(testing_asXY, "EBK", ID)

In [15]:
raster = toRaster(ebk_totest, "EBKRas", ID)

In [16]:
# testing_spatjoin = density(testing_asXY, radius, "count", ID) (you never really use this)
validation_spatjoin = density(validation_asXY, testing_asXY, radius, "valid_with_count", ID) # returns Join_Count column in feature class

In [17]:
toAnalyze = ExtractToPoints(validation_spatjoin, raster, "Analysis", ID)

In [18]:
# makes df from feature class, edits columns to include only graphs
analysis = pd.DataFrame.spatial.from_featureclass(toAnalyze) 
analysis.rename(columns={'RASTERVALU': 'predicted_pH'}, inplace=True)
analysis['error'] = analysis['ph_h2o']-analysis['predicted_pH']
analysis = analysis.filter(items = ['Join_Count', 'error'])
# export
# analysis.to_csv(r"C:\path_to_your.csv", index=False)

In [ ]:
# convert to numpy array
num1 = analysis.to_numpy()
join_count = np.unique(num1[:, 0])

counts = {}
alldata = []
for i, val in enumerate(join_count):
    a = num1[num1[:, 0] == val, 1]
    b = a[~pd.isna(a)]
    c = np.abs(b)
    alldata.append(np.array(c, dtype=float))
    counts[val] = len(a)

# plots
fig, ax = plt.subplots(figsize=(9, 4))

# plot box plot
boxplot = ax.boxplot(alldata, sym="k.")
ax.set_title('Density vs Error', pad = 20)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.grid(True)
ax.set_xticks([y + 1 for y in range(len(alldata))], labels=join_count)
ax.set_xlabel('# of points in radius ' + str(radius) + ' km')
ax.set_ylabel('Error')

max_y = max([whisker.get_ydata()[1] for whisker in boxplot['whiskers'][1::2]]) 

# Annotate the box plot with the number of points
annotation_y = max_y + (max_y * 0.25)
for i, (val, count) in enumerate(counts.items()):
    ax.annotate(f'{count}', xy=(i + 1, annotation_y), xytext=(0, 0),
                textcoords='offset points', ha='center', va='bottom', fontsize=9)

plt.show()
